In [ ]:
import pandas as pd
import numpy as np
import re

# Leggi il file CSV
df = pd.read_csv('/Users/edoardoconti/Downloads/run befana/results_completo_cleaned.csv')
# df = pd.read_csv('/Users/edoardoconti/Downloads/ultima run/results_pretrained.csv')
# df = pd.read_csv('/Users/edoardoconti/Downloads/run 22 gen /results.csv')


# Estrai il tipo di esperimento da 'experiment'
#df['experiment_type'] = df['experiment'].apply(lambda x: re.search(r'^(obd|clm|resnet)', x).group(1) if re.search(r'^(obd|clm|resnet)', x) else 'unknown')
df['experiment_type'] = df['experiment'].apply(lambda x: re.search(r'_(obd|clm|resnet18|resnetQWK)', x).group(1) if re.search(r'_(obd|clm|resnet18|resnetQWK)', x) else 'unknown')

df['experiment_type'] = df['experiment_type'].replace('resnet18', 'resnet18 CCE')
df['experiment_type'] = df['experiment_type'].replace('resnetQWK', 'resnet18 QWK')

# Definisci le metriche da calcolare
#metrics = ['ccr', 'f1', 'acc_1off', 'qwk', 'mae', 'ms', 'rmse']
metrics = ["ccr", "f1", "acc_1off", "acc_2off", "qwk", "spearman", "ms", "mae", "rmse"]

# Funzione per calcolare media e deviazione standard delle metriche
def calculate_stats(experiment_df):
    mean_values = experiment_df[metrics].mean()
    std_values = experiment_df[metrics].std()
    return mean_values, std_values

# Calcola media e deviazione standard per ogni tipo di esperimento
experiment_types = df['experiment_type'].unique()

# Stampa le medie e le deviazioni standard per ogni tipo di esperimento
for exp_type in experiment_types:
    exp_df = df[df['experiment_type'] == exp_type]
    mean_values, std_values = calculate_stats(exp_df)
    
    print(f"\nExperiment Type: {exp_type}")
    print("Mean Values:")
    print(mean_values)
    print("\nStandard Deviation Values:")
    print(std_values)

In [ ]:
import matplotlib.pyplot as plt

for metric in metrics:
    plt.figure(figsize=(8, 4))  # Aumenta la larghezza della figura per dar spazio all'etichetta dell'asse x
    positions = np.arange(len(experiment_types))
    positions = positions[::-1] #reverse
    # positions[2], positions[3] = positions[3], positions[2]

    medianprops = dict(linestyle='-', color='black')
    
    # Creazione del boxplot con colori
    box_plot = plt.boxplot([df[df['experiment_type'] == exp_type][metric] for exp_type in experiment_types], 
                           positions=positions, 
                           labels=[exp_type.upper() for exp_type in experiment_types],  # Rende maiuscole le etichette
                           showfliers=False,
                           vert=False,
                           widths=0.65,
                           patch_artist=True,
                           medianprops=medianprops
                           )
    
    # Specifica i colori e le gradazioni
    colors = plt.cm.YlGn(np.linspace(0.1, 0.5, len(experiment_types)))
    # colors = plt.cm.YlGn(np.linspace(0.4, 0.1, len(experiment_types)))
    
    # Assegna colori ai boxplot
    for i, box in enumerate(box_plot['boxes']):
        box.set_facecolor(colors[i])
    
    # Modifica la dimensione del font per le etichette
    plt.xlabel(metric.upper(), fontsize=20, labelpad=15)  # Aggiunge spazio sotto l'etichetta dell'asse x
    
    # Modifica la dimensione del font per le etichette degli assi
    plt.tick_params(axis='both', labelsize=15)
    
    plt.show()